## Initialize

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

## Import the data

In [5]:
dataset = pd.read_csv("diagnosis.data", header = None, delimiter = r"\s+")
dataset.shape

(120, 8)

In [6]:
dataset.head()

,0,1,2,3,4,5,6,7
0,35.5,no,yes,no,no,no,no,no
1,35.9,no,no,yes,yes,yes,yes,no
2,35.9,no,yes,no,no,no,no,no
3,36.0,no,no,yes,yes,yes,yes,no
4,36.0,no,yes,no,no,no,no,no


## Create X and Y

In [7]:
X = dataset.iloc[:, 0:6].values
Y1 = dataset.iloc[:, 6:7].values
Y2 = dataset.iloc[:, 7:8].values

In [8]:
X.shape

(120, 6)

In [10]:
Y1.shape

(120, 1)

In [11]:
Y2.shape

(120, 1)

## Preprocess the Data

In [13]:
le_Y = LabelEncoder()
Y1 = le_Y.fit_transform(Y1)
Y2 = le_Y.transform(Y2)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [14]:
Y1[0:5]

array([0, 1, 0, 1, 0], dtype=int64)

In [15]:
Y2[0:5]

array([0, 0, 0, 0, 0], dtype=int64)

In [16]:
Y1 = Y1.reshape(len(Y1), 1)
Y2 = Y2.reshape(len(Y2), 1)

In [17]:
ohe_Y = OneHotEncoder(categorical_features = [0])

In [18]:
Y1 = ohe_Y.fit_transform(Y1).toarray()
Y1[0]

array([ 1.,  0.])

In [19]:
Y2 = ohe_Y.transform(Y2).toarray()
Y2[0]

array([ 1.,  0.])

In [20]:
def encoder_X(index):
    le_X = LabelEncoder()
    X[:, index] = le_X.fit_transform(X[:, index])

In [21]:
for i in range(1, 6):
    encoder_X(i)

In [22]:
sc_X = StandardScaler()

In [23]:
X = sc_X.fit_transform(X)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [24]:
X[0]

array([-1.77979629, -0.56451866,  0.84515425, -1.41421356, -0.98346994,
       -0.84515425])

## Create Train and Test Data

In [26]:
X1_train, X1_test, Y1_train, Y1_test = train_test_split(X, Y1, test_size = 0.2, random_state = 4)
X2_train, X2_test, Y2_train, Y2_test = train_test_split(X, Y2, test_size = 0.2, random_state = 4)

## Create TF Classifier

In [27]:
num_features = X.shape[1]
num_features

6

In [29]:
num_classes = Y1.shape[1]
num_classes

2

In [30]:
# Y = W1.X1 + W2.X2 + ... + Wn.Xn + B
# output = 1 / (1 + e ^ (- Y))
# Create Weights and Biases Variables

W = tf.Variable(tf.zeros([num_features, num_classes]))
B = tf.Variable(tf.zeros([num_classes]))

In [31]:
# Create x and y_ placeholders to hold actual data
x = tf.placeholder(tf.float32, [None, num_features])
y_ = tf.placeholder(tf.float32, [None, num_classes])

In [32]:
# Calculate y which holds the model's predicted values
Wx = tf.matmul(x, W)
y = Wx + B

In [33]:
# Create a cost function to minimize
cost_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y, labels = y_))

In [34]:
# Create an Optimizer to minimize the cost
# Use different Optimizers to compare which suits the given problem
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.5).minimize(cost_cross_entropy)

In [36]:
def trainTheData(num_steps, optimizer_to_use, batch_size, X_train, Y_train, X_test):
    init = tf.global_variables_initializer()
    # initialize the global variables
    
    with tf.Session() as sess:
        sess.run(init)
        
        for i in range(num_steps):
            
            # Calculate the batch start index
            if batch_size == len(X_train):
                batch_start_index = 0
            elif batch_size > len(X_train):
                raise ValueError("Batch Size : " + str(batch_size) + " cannot be greater than Data Size : ", len(X_train))
            else:
                batch_start_index = (i * batch_size) % (len(X_train) - batch_size)
                
            # Calculate the batch end index
            batch_end_index = batch_size + batch_start_index
            
            # Calculate the batch X and Y Values
            batch_X_values = X_train[batch_start_index : batch_end_index]
            batch_Y_values = Y_train[batch_start_index : batch_end_index]
            
            # Create feed dict to feed it to the optimizer
            feed = {x : np.array(batch_X_values), y_ : np.array(batch_Y_values)}
            
            sess.run(optimizer_to_use, feed_dict = feed)
            
            # Print the metrics and other parameters for every 25th iteration
            if (i + 1) % 25 == 0:
                print("After " + str(i + 1) + " iterations, cost : ", sess.run(cost_cross_entropy, feed_dict = feed))
                print("W : ", sess.run(W))
                print("B : ", sess.run(B))
                print("")
        
        # Calculate the model's predicted logit values
        Y_pred_logits = sess.run(y, feed_dict = {x : np.array(X_test)})
        
        # Convert the logit values to a tensor
        Y_pred_logits_tensor = tf.convert_to_tensor(Y_pred_logits)
        
        # Apply softmax function
        apply_softmax = tf.nn.softmax(Y_pred_logits_tensor)
        
        # Calculate the model's predicted classes
        Y_pred_classes = np.argmax(sess.run(apply_softmax), axis = 1)
        
        sess.close()
    return Y_pred_classes   

In [40]:
Y1_pred_classes = trainTheData(num_steps = 200, optimizer_to_use = optimizer, batch_size = len(X1_train), X_train = X1_train, Y_train = Y1_train, X_test = X1_test)

After 25 iterations, cost :  0.0635387
W :  [[ 0.36048052 -0.36048049]
 [-0.35461965  0.35461956]
 [ 0.74408579 -0.74408567]
 [-1.0390079   1.03900814]
 [-0.71595895  0.71595913]
 [ 0.17889084 -0.17889084]]
B :  [ 0.00907903 -0.00907903]

After 50 iterations, cost :  0.0358602
W :  [[ 0.42804849 -0.42804846]
 [-0.47170502  0.47170499]
 [ 0.94159371 -0.94159365]
 [-1.33542526  1.33542538]
 [-0.85805929  0.85805947]
 [ 0.24417518 -0.2441752 ]]
B :  [ 0.0272304  -0.02723039]

After 75 iterations, cost :  0.0252447
W :  [[ 0.46254206 -0.46254203]
 [-0.54670739  0.54670739]
 [ 1.06801081 -1.06801069]
 [-1.52046967  1.5204699 ]
 [-0.94497192  0.9449721 ]
 [ 0.27927175 -0.27927178]]
B :  [ 0.04395306 -0.04395306]

After 100 iterations, cost :  0.0195487
W :  [[ 0.48453444 -0.48453441]
 [-0.60238218  0.60238218]
 [ 1.16253662 -1.1625365 ]
 [-1.65579653  1.65579689]
 [-1.00820744  1.00820768]
 [ 0.30259895 -0.30259901]]
B :  [ 0.05837702 -0.05837702]

After 125 iterations, cost :  0.0159759
W :

In [41]:
Y1_test_classes = np.argmax(Y1_test, axis = 1)

In [44]:
Y2_pred_classes = trainTheData(num_steps = 200, optimizer_to_use = optimizer, batch_size = len(X2_train), X_train = X2_train, Y_train = Y2_train, X_test = X2_test)

After 25 iterations, cost :  0.0406125
W :  [[-0.75100392  0.75100386]
 [-0.63314885  0.63314891]
 [-0.89944923  0.89944935]
 [-0.38147601  0.38147599]
 [ 0.0063748  -0.00637479]
 [-0.48020461  0.48020458]]
B :  [ 0.17663513 -0.17663509]

After 50 iterations, cost :  0.0225144
W :  [[-0.90464759  0.90464747]
 [-0.76825464  0.76825464]
 [-1.10405803  1.10405815]
 [-0.49179092  0.49179086]
 [ 0.03844102 -0.03844102]
 [-0.59587872  0.59587878]]
B :  [ 0.20157337 -0.20157333]

After 75 iterations, cost :  0.0158045
W :  [[-0.99843419  0.99843413]
 [-0.85430014  0.85430014]
 [-1.22835743  1.22835755]
 [-0.55999815  0.5599981 ]
 [ 0.05868994 -0.05868995]
 [-0.66576862  0.66576868]]
B :  [ 0.21472223 -0.21472217]

After 100 iterations, cost :  0.0122537
W :  [[-1.0666008   1.06660092]
 [-0.91837496  0.91837496]
 [-1.31826305  1.31826305]
 [-0.6099273   0.6099273 ]
 [ 0.07363681 -0.07363681]
 [-0.7162773   0.71627742]]
B :  [ 0.22351944 -0.22351936]

After 125 iterations, cost :  0.0100409
W :

In [45]:
Y2_test_classes = np.argmax(Y2_test, axis = 1)

## Check the accuracy

In [42]:
accuracy_score(Y1_test_classes, Y1_pred_classes)

1.0

In [43]:
confusion_matrix(Y1_test_classes, Y1_pred_classes)

array([[13,  0],
       [ 0, 11]], dtype=int64)

In [46]:
accuracy_score(Y2_test_classes, Y2_pred_classes)

1.0

In [47]:
confusion_matrix(Y2_test_classes, Y2_pred_classes)

array([[15,  0],
       [ 0,  9]], dtype=int64)